In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `bootpython && heroku run --app guarded-everglades-89687 make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run --app guarded-everglades-89687 make importdb && make pipeline`
# (6) Remember to delete contents of ~/Documents/links.txt
links = pd.read_csv('data/export.csv')
links

/var/folders/7f/6ty6wfcd1hs3d1wz9b1c4gsw0000gn/T/ipykernel_10019/1024356683.py:12: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108358,172851,https://www.scotthyoung.com/blog/2020/05/08/la...,Macedonian Project (Week Three Update) - Scott...,\N,\N,2024-01-12 10:46:32.465470,2024-01-12 10:46:32.465474,\N,\N,Custom,84,0,\N
108359,172852,https://twitter.com/messages/25776739-54765863,Aaron Kreider / X,\N,\N,2024-01-12 10:46:32.471538,2024-01-12 10:46:32.471543,\N,\N,Custom,86,0,\N
108360,172853,https://www.wikiwand.com/en/List_of_countries_...,List of countries and dependencies by populati...,\N,\N,2024-01-12 10:46:32.477460,2024-01-12 10:46:32.477465,\N,\N,Custom,70,0,\N
108361,172854,https://www.lesswrong.com/inbox/LHQekBPbhDiJqAWju,Private Conversation — LessWrong,\N,\N,2024-01-12 10:46:32.483525,2024-01-12 10:46:32.483530,\N,\N,Custom,27,0,\N


In [2]:
links['added'].max()

'2024-01-12 10:46:32.489508'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
108358    172851
108359    172852
108360    172853
108361    172854
108362    172855
Name: id, Length: 15662, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
108286,172779,https://www.cnas.org/research/technology-and-n...,Center for a New American Security (en-US),\N,\N,2024-01-12 10:46:32.052707,2024-01-12 10:46:32.052712,\N,\N,Custom,34,0,\N
108287,172780,https://forum.effectivealtruism.org/posts/KYzb...,Project proposal: Scenario analysis group for ...,\N,\N,2024-01-12 10:46:32.058383,2024-01-12 10:46:32.058387,\N,\N,Custom,99,0,\N
108290,172783,https://www.lesswrong.com/posts/keiYkaeoLHoKK4...,Six Dimensions of Operational Adequacy in AGI ...,\N,\N,2024-01-12 10:46:32.075573,2024-01-12 10:46:32.075578,\N,\N,Custom,23,0,\N
108291,172784,https://www.lesswrong.com/posts/uMQ3cqWDPHhjti...,AGI Ruin: A List of Lethalities — LessWrong,\N,\N,2024-01-12 10:46:32.081272,2024-01-12 10:46:32.081277,\N,\N,Custom,47,0,\N
108313,172806,https://tellingthefuture.substack.com/p/what-k...,What Kind of Future Will AI Bring? - by Robert...,\N,\N,2024-01-12 10:46:32.207342,2024-01-12 10:46:32.207346,\N,\N,Custom,73,0,\N
108314,172807,https://tellingthefuture.substack.com/p/foreca...,Forecasting the End of the World - by Robert d...,\N,\N,2024-01-12 10:46:32.212988,2024-01-12 10:46:32.212993,\N,\N,Custom,70,0,\N
108315,172808,https://www.facebook.com/messages/t/547821577/,Messenger,\N,\N,2024-01-12 10:46:32.218756,2024-01-12 10:46:32.218760,\N,\N,Custom,38,0,\N
108319,172812,https://joecarlsmith.com/2024/01/09/does-ai-ri...,Does AI risk “other” the AIs? - Joe Carlsmith,\N,\N,2024-01-12 10:46:32.242078,2024-01-12 10:46:32.242082,\N,\N,Custom,38,0,\N
108323,172816,https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.1...,Artificial Intelligence Risk Management Framew...,\N,\N,2024-01-12 10:46:32.265347,2024-01-12 10:46:32.265352,\N,\N,Custom,1,0,\N
108326,172819,https://www.rand.org/pubs/working_papers/WRA28...,Securing Artificial Intelligence Model Weights...,\N,\N,2024-01-12 10:46:32.282625,2024-01-12 10:46:32.282630,\N,\N,Custom,98,0,\N


In [5]:
links[links['url'].duplicated()]['id']

108286    172779
108287    172780
108290    172783
108291    172784
108313    172806
108314    172807
108315    172808
108319    172812
108323    172816
108326    172819
108334    172827
108338    172831
108344    172837
108353    172846
108356    172849
108357    172850
108358    172851
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/122
...11/122
...21/122
...31/122
...41/122
...51/122
...61/122
...71/122
...81/122
...91/122
...101/122
...111/122
...121/122
...122/122


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108479,172972,https://www.scotthyoung.com/blog/2020/05/08/la...,Macedonian Project (Week Three Update) - Scott...,\N,\N,2024-01-12 10:47:47.194037,2024-01-12 10:47:47.194041,\N,\N,Custom,61,0,\N
108480,172973,https://twitter.com/messages/25776739-54765863,Aaron Kreider / X,\N,\N,2024-01-12 10:47:47.199839,2024-01-12 10:47:47.199844,\N,\N,Custom,60,0,\N
108481,172974,https://www.wikiwand.com/en/List_of_countries_...,List of countries and dependencies by populati...,\N,\N,2024-01-12 10:47:47.205996,2024-01-12 10:47:47.206001,\N,\N,Custom,71,0,\N
108482,172975,https://www.lesswrong.com/inbox/LHQekBPbhDiJqAWju,Private Conversation — LessWrong,\N,\N,2024-01-12 10:47:47.211929,2024-01-12 10:47:47.211934,\N,\N,Custom,86,0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10,0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9,0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85,0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82,0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108479,172972,https://www.scotthyoung.com/blog/2020/05/08/la...,Macedonian Project (Week Three Update) - Scott...,\N,\N,2024-01-12 10:47:47.194037,2024-01-12 10:47:47.194041,\N,\N,Custom,61,0,\N
108480,172973,https://twitter.com/messages/25776739-54765863,Aaron Kreider / X,\N,\N,2024-01-12 10:47:47.199839,2024-01-12 10:47:47.199844,\N,\N,Custom,60,0,\N
108481,172974,https://www.wikiwand.com/en/List_of_countries_...,List of countries and dependencies by populati...,\N,\N,2024-01-12 10:47:47.205996,2024-01-12 10:47:47.206001,\N,\N,Custom,71,0,\N
108482,172975,https://www.lesswrong.com/inbox/LHQekBPbhDiJqAWju,Private Conversation — LessWrong,\N,\N,2024-01-12 10:47:47.211929,2024-01-12 10:47:47.211934,\N,\N,Custom,86,0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108479,172972,https://www.scotthyoung.com/blog/2020/05/08/la...,Macedonian Project (Week Three Update) - Scott...,\N,\N,2024-01-12 10:47:47.194037,2024-01-12 10:47:47.194041,\N,\N,Custom,61,0,\N
108480,172973,https://twitter.com/messages/25776739-54765863,Aaron Kreider / X,\N,\N,2024-01-12 10:47:47.199839,2024-01-12 10:47:47.199844,\N,\N,Custom,60,0,\N
108481,172974,https://www.wikiwand.com/en/List_of_countries_...,List of countries and dependencies by populati...,\N,\N,2024-01-12 10:47:47.205996,2024-01-12 10:47:47.206001,\N,\N,Custom,71,0,\N
108482,172975,https://www.lesswrong.com/inbox/LHQekBPbhDiJqAWju,Private Conversation — LessWrong,\N,\N,2024-01-12 10:47:47.211929,2024-01-12 10:47:47.211934,\N,\N,Custom,86,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)